## T-distributed Stochastic Neighbor Embedding

### Joint Probability in High Dimensional Space

Given a set of $n$ high-dimensional objects $\mathbf{x}_1, \ldots, \mathbf{x}_n$, t-SNE first computes joint probabilities $p_{i j}$, represented by a symmetric matrix $\mathbf{P}=\left(p_{i j}\right)_{1 \leq i, j \leq n} \in \mathbb{R}^{n \times n}$, that are proportional to the similarity of objects $\mathbf{x}_i$ and $\mathbf{x}_j$. The t-SNE method models the joint probabilities as a **Gaussian distribution** around each data points in the higher dimensional space.
For $i \neq j$, define
$$
p_{j \mid i}=\frac{\exp \left(-\left\|\mathbf{x}_i-\mathbf{x}_j\right\|^2 / 2 \sigma_i^2\right)}{\sum_{k \neq i} \exp \left(-\left\|\mathbf{x}_i-\mathbf{x}_k\right\|^2 / 2 \sigma_i^2\right)}
$$
and set $p_{i \mid i}=0$ (a data point's joint probability with itself). Note that $\sum_j p_{j \mid i}=1$ for all $i$ by the definition of conditional probability; in other words, for each data point $i$ (a n-dimensional vector), there must be $j$ partitions of the sample space $p_{1\mid i} + p_{2\mid i} ... + p_{j\mid i} + p_{n-1\mid i} + 0=1$. As Van der Maaten and Hinton explained: 

> "The similarity of datapoint $x_j$ to datapoint $x_i$ is the conditional probability, $p_{j \mid i}$, that $x_i$ would pick $x_j$ as its neighbor if neighbors were picked in proportion to their probability density under a Gaussian centered at $x_i$.

  * In the formula above, the bandwidth of the Gaussian kernels $\sigma_i$ is set in such a way that the entropy of the conditional distribution equals a predefined entropy using the bisection method. As a result, the bandwidth is adapted to the density of the data: smaller values of $\sigma_i$ are used in denser parts of the data space. Intuitively, the value set for perplexity $\mathcal{P}$ can be interpreted as a smooth measure of the effective number of neighbors; it takes values typically between 5 and 50. Modern implementations construct a KNN graph with $k=3\mathcal{P}$ and only consider affinities between connected nodes as non-zero. The default perplexity value in most implementations is $\mathcal{P}=30$.
  $$\mathcal{P}=2^{\mathcal{H}} \text{, where } \mathcal{H}=-\sum_{j \neq i} p_{j \mid i} \log _2 p_{j \mid i} \text{ is the entropy}$$

  * Since the Gaussian kernel uses the Euclidean distance $\left\|x_i-x_j\right\|$, it is affected by the curse of dimensionality; in high dimensional data, when distances lose the ability to discriminate, the $p_{i j}$ become too similar (asymptotically, they would converge to a constant). It has been proposed to adjust the distances with a power transform, based on the intrinsic dimension of each point, to alleviate this.

Since conditional probabilities may not be symmetric, we symmetrize $p_{j \mid i}$ and $p_{i \mid j}$ as follow:
$$
p_{i j}=\frac{p_{j \mid i}+p_{i \mid j}}{2 n}
$$
We normalize by $2n$ so that the entire matrix $P$ sums to 1.

<hr/>

### Joint Probability in Lower Dimensional Space

The t-SNE aims to learn a $d$-dimensional map $\mathbf{y}_1, \ldots, \mathbf{y}_n$ (with $\mathbf{y}_i \in \mathbb{R}^d$ and $d$ typically chosen as 2 or 3 ) that reflects the similarities $p_{i j}$ as well as possible. To this end, it measures similarities $q_{i j}$, represented by a symmetric matrix $\mathbf{Q}=\left(q_{i j}\right)_{1 \leq i, j \leq n} \in \mathbb{R}^{d \times d}$, between two points in the map $\mathbf{y}_i$ and $\mathbf{y}_j$, using a very similar approach. Specifically, for $i \neq j$, define $q_{i j}$ as
$$
q_{i j}=\frac{\left(1+\left\|\mathbf{y}_i-\mathbf{y}_j\right\|^2\right)^{-1}}{\sum_{k \neq l}\left(1+\left\|\mathbf{y}_k-\mathbf{y}_l\right\|^2\right)^{-1}}
$$
and set $q_{i i}=0$. Herein a heavy-tailed Student t-distribution (with one-degree of freedom, which is the same as a Cauchy distribution) is used to measure similarities between low-dimensional points in order to allow dissimilar objects to be modeled far apart in the map. For convenience, we can let

$$q_{i j}=\frac{w_{i j}}{Z}, \quad w_{i j}=(1+d_{i j}^2)^{-1}, \quad d_{i j}=\left\|\mathbf{y}_i-\mathbf{y}_j\right\|, \quad Z=\sum_{k \neq l} w_{k l}$$

<hr/>

### Optimization

The locations of the points $\mathbf{y}_i$ in the map are determined by minimizing the (non-symmetric) Kullback-Leibler divergence of the distribution $P$ from the distribution $Q$, that is:
$$
\left(\mathbf{y}_1, \ldots, \mathbf{y}_n\right)=\underset{\mathbf{y}_1, \ldots, \mathbf{y}_n}{\arg \min } \; D_{K L}(\mathbf{P}, \mathbf{Q})=\underset{\substack{\mathbf{y}_1, \ldots, \mathbf{y}_n}}{\arg \min } \sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log \frac{p_{i j}}{q_{i j}}
$$
The minimization of the Kullback-Leibler divergence with respect to the points $\mathbf{y}_i$ is performed using gradient descent. The result of this optimization is a map that reflects the similarities between the high-dimensional inputs. 

&nbsp;
&nbsp;

We can use the identity $\log \frac{a}{b} = \log a - \log b$ so

$$
\begin{align*}
\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log \frac{p_{i j}}{q_{i j}} &= \sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} (\log p_{i j} - \log q_{ij}) \\ 
&= \sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} \textcolor{blue}{p_{i j}\log p_{i j}} - p_{i j}\log q_{ij} \\ 
&= \sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} 0 - p_{i j}\log q_{ij} \\ 
&= -\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j}\log q_{ij} 
\end{align*}
$$

The first term of the summation in blue is a constant value that does not depend on the low-dimensional representation $q_{ij}$. It does not affect the optimization process, as the algorithm is trying to find the optimal low-dimensional representation $q_{ij}$, which involve the variables $\mathbf{y}_1, \ldots, \mathbf{y}_n$, that minimizes the KL divergence. So the above expression can be simplified to:

$$\underset{\substack{\mathbf{y}_1, \ldots, \mathbf{y}_n}}{\arg \min } \quad -\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j}\log q_{ij}$$

Substituting $w_{i j}$ and $Z$ for $q_{ij}$, the loss function becomes:

$$
\begin{align*}
-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log \frac{w_{i j}}{Z}&=-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} (\log w_{i j} - \log Z) \\
&=-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log w_{i j}+p_{i j}\log \sum_{i \neq j} w_{i j} \\
&=-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log w_{i j}+\textcolor{blue}{\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}}p_{i j}}\log \sum_{i \neq j} w_{i j} \\
&=-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log w_{i j}+\textcolor{blue}{1}\log \sum_{i \neq j} w_{i j} \\ 
&=-\sum_{\substack{i, j \in\{1,2, \ldots, n\} \\ i \neq j}} p_{i j} \log w_{i j}+\log \sum_{i \neq j} w_{i j}
\end{align*}
$$

Finally, the loss function can be expressed as follows:

$$\underset{\substack{\mathbf{y}_1, \ldots, \mathbf{y}_n}}{\arg \min } \quad \textcolor{blue}{-\sum_{i \neq j} p_{i j} \log w_{i j}}+\textcolor{red}{\log \sum_{i \neq j} w_{i j}}$$

This reformulated as an N-body simulation problem with the first term in red as the attraction force and the second term in blue as the repulsive forces. Using $\partial w_{i j} / \partial \mathbf{y}_i=$ $-2 w_{i j}^2\left(\mathbf{y}_i-\mathbf{y}_j\right)$, the gradient can be written as

$$
\begin{aligned} \frac{\partial \mathcal{L}_{\mathrm{t}-\mathrm{SNE}}}{\partial \mathbf{y}_i} & =-2 \sum_j p_{i j} \frac{1}{w_{i j}} \frac{\partial w_{i j}}{\partial \mathbf{y}_i}+2 \frac{1}{Z} \sum_j \frac{\partial w_{i j}}{\partial \mathbf{y}_i} \\ & \sim \sum_j p_{i j} w_{i j}\left(\mathbf{y}_i-\mathbf{y}_j\right)-\frac{1}{Z} \sum_j w_{i j}^2\left(\mathbf{y}_i-\mathbf{y}_j\right)\end{aligned}
$$

<hr/>

### Sklearn TSNE

In [ ]:
from sklearn.manifold import TSNE

TSNE(
    n_components=2, # Usually 2 or 3
    perplexity=30.0, # Usually between 5 and 50
    early_exaggeration=12.0, # Controls how tight natural clusters in the original space are in the embedded space and how much space will be between them; choice of this parameter is not very critical
    learning_rate='auto', # A heuristic suggested in Belkina et al. (2019) is to set the learning rate to the sample size divided by the early exaggeration factor
    n_iter=1000, # Number of iterations for the optimization, at least 250
    metric='euclidean', 
    init='pca', # Initialization of embedding--- pca: PCA initialization (first two components of PCA are used to initialize the embedding), random: random initialization
    random_state=None, # Note that different initializations might result in different local minima of the cost function
    method='barnes_hut', # Barnes-Hut approximation running in O(NlogN) time; the exact method is more exact but cannot scale to millions of examples
    angle=0.5, # This is used if method=’barnes_hut’, but the method is not very sensitive to changes in this parameter in the range of 0.2 - 0.8
    n_jobs=-1
)

### Cuml TSNE

In [ ]:
from cuml import TSNE 

TSNE(
    n_components=2, # Only 2 is supported
    perplexity=30.0, 
    early_exaggeration=12.0, # Not critical to tune this
    late_exaggeration=1.0, # May be beneficial to increase this slightly to improve cluster separation
    learning_rate=200.0, # The learning rate usually between (10, 1000)
    n_iter=1000, # The more epochs, the more stable/accurate the final embedding
    metric='euclidean', # [‘l1, ‘cityblock’, ‘manhattan’, ‘euclidean’, ‘l2’, ‘sqeuclidean’, ‘minkowski’, ‘chebyshev’, ‘cosine’, ‘correlation’]
    metric_params=None, 
    init='random', # Currently only random initialization is supported
    verbose=False, 
    random_state=None, # Setting this can make repeated runs look more similar. Note, however, that this highly parallelized t-SNE implementation is not completely deterministic between runs, even with the same random_state
    method='fft', # [‘exact’, ‘barnes_hut’, ‘fft’] ‘barnes_hut’ and ‘fft’ are fast approximations, and ‘exact’ is more accurate but slower
    angle=0.5, # This is used if method=’barnes_hut’, but the method is not very sensitive to changes in this parameter in the range of 0.2 - 0.8
    learning_rate_method='adaptive', # The ‘adaptive’ method tunes the learning rate, early exaggeration, perplexity and n_neighbors automatically based on input size
    n_neighbors=90, # The number of data points to use in the attractive forces. Smaller values are better for preserving local structure, whilst larger values can improve global structure preservation
    perplexity_max_iter=100, # The number of epochs the best gaussian bands are found for
    exaggeration_iter=250, # To promote the growth of clusters, set this higher
    pre_momentum=0.5, # During the exaggeration iteration, more forcefully apply gradients
    post_momentum=0.8, # During the late phases, less forcefully apply gradients
    square_distances=True, # This argument should likely be set to False for distance metrics other than ‘euclidean’ and ‘l2’
    handle=None, 
    output_type=None
)

For visualizing the effects of different parameters, see the following blog "[How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/)".

## T-SNE FAQ

This list of FAQs is obtained from Laurens van der Maaten's [website](https://lvdmaaten.github.io/tsne/#faq):


* How can I asses the quality of the visualizations that t-SNE constructed?

    Preferably, just look at them! Notice that t-SNE does not retain distances but probabilities, so measuring some error between the Euclidean distances in high-D and low-D is useless. However, if you use the same data and perplexity, you can compare the Kullback-Leibler divergences that t-SNE reports. It is perfectly fine to run t-SNE ten times, and select the solution with the lowest KL divergence.

* How should I set the perplexity in t-SNE?

    The performance of t-SNE is fairly robust under different settings of the perplexity. The most appropriate value depends on the density of your data. Loosely speaking, one could say that a larger / denser dataset requires a larger perplexity. Typical values for the perplexity range between 5 and 50.


* What is perplexity anyway?

    Perplexity is a measure for information that is defined as 2 to the power of the Shannon entropy. The perplexity of a fair die with k sides is equal to k. In t-SNE, the perplexity may be viewed as a knob that sets the number of effective nearest neighbors. It is comparable with the number of nearest neighbors k that is employed in many manifold learners.


* Every time I run t-SNE, I get a (slightly) different result?

    In contrast to, e.g., PCA, t-SNE has a non-convex objective function. The objective function is minimized using a gradient descent optimization that is initiated randomly. As a result, it is possible that different runs give you different solutions. Notice that it is perfectly fine to run t-SNE a number of times (with the same data and parameters), and to select the visualization with the lowest value of the objective function as your final visualization.


* When I run t-SNE, I get a strange ‘ball’ with uniformly distributed points?

    This usually indicates you set your perplexity way too high. All points now want to be equidistant. The result you got is the closest you can get to equidistant points as is possible in two dimensions. If lowering the perplexity doesn’t help, you might have run into the problem described in the next question. Similar effects may also occur when you use highly non-metric similarities as input.


* When I run t-SNE, it reports a very low error but the results look crappy?

    Presumably, your data contains some very large numbers, causing the binary search for the correct perplexity to fail. In the beginning of the optimization, t-SNE then reports a minimum, mean, and maximum value for sigma of 1. This is a sign that something went wrong! Just divide your data or distances by a big number, and try again.


* I tried everything you said, but t-SNE still doesn’t seem to work very well?

    Maybe there is something weird in your data. As a sanity check, try running PCA on your data to reduce it to two dimensions. If this also gives bad results, then maybe there is not very much nice structure in your data in the first place. If PCA works well but t-SNE doesn’t, I am fairly sure you did something wrong. Just check your code again until you found the bug! If nothing works, feel free to drop me a line.


* Can I use a pairwise Euclidean distance matrix as input into t-SNE?

    Yes you can! Download the Matlab implementation, and use your pairwise Euclidean distance matrix as input into the tsne_d.m function.


* Can I use a pairwise similarity matrix as input into t-SNE?

    Yes you can! For instance, we successfully applied t-SNE on a dataset of word association data. Download the Matlab implementation, make sure the diagonal of the pairwise similarity matrix contains only zeros, symmetrize the pairwise similarity matrix, and normalize it to sum up to one. You can now use the result as input into the tsne_p.m function.


* Can I use t-SNE to embed data in more than two dimensions?

    Well, yes you can, but there is a catch. The key characteristic of t-SNE is that it solves a problem known as the crowding problem. The extent to which this problem occurs depends on the ratio between the intrinsic data dimensionality and the embedding dimensionality. So, if you embed in, say, thirty dimensions, the crowding problem is less severe than when you embed in two dimensions. As a result, it often works better if you increase the degrees of freedom of the t-distribution when embedding into thirty dimensions (or if you try to embed intrinsically very low-dimensional data such as the Swiss roll). More details about this are described in the [AI-STATS paper](https://lvdmaaten.github.io/publications/papers/AISTATS_2009.pdf).


* Why doesn’t t-SNE work as well as LLE or Isomap on the Swiss roll data?

    When embedding the Swiss roll data, the crowding problem does not apply. So you may have to use a lighter-tailed t-distribution to embed the Swiss toll successfully (see above). But frankly… who cares about Swiss rolls when you can embed complex real-world data nicely?


* Once I have a t-SNE map, how can I embed incoming test points in that map?

    t-SNE learns a non-parametric mapping, which means that it does not learn an explicit function that maps data from the input space to the map. Therefore, it is not possible to embed test points in an existing map (although you could re-run t-SNE on the full dataset). A potential approach to deal with this would be to train a multivariate regressor to predict the map location from the input data. Alternatively, you could also make such a regressor minimize the t-SNE loss directly, which is what I did in this [paper](https://lvdmaaten.github.io/publications/papers/AISTATS_2009.pdf).